In [1]:
test_input = """        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5"""

In [2]:
import itertools
import numpy as np
from collections import defaultdict
from collections_extended import bijection

In [3]:
def parse_p1(s):
    board, code = s.split("\n\n")
    board_lines = board.split("\n")

    num_rows = len(board_lines)
    num_cols = max(len(b) for b in board_lines)
    num_rows, num_cols


    max_line = max(len(b) for b in board_lines)

    # position, is_open
    tiles = dict()

    for i, l in enumerate(board_lines):
        for j, c in enumerate(l):
            p = (i, j)
            if c == ".":
                tiles[p] = True
            elif c == '#':
                tiles[p] = False
            else:
                pass

    just_tiles = list(tiles.keys())
    bounds = []

    for axis in [0, 1]:
        axis_bounds = []
        k = lambda x: x[axis]
        for i, g in itertools.groupby(sorted(just_tiles, key=k), key=k):
            group = list(g)

            other_axis = 1-axis
            other_axis_vals = [e[other_axis] for e in group]

            axis_bounds.append((min(other_axis_vals), max(other_axis_vals)))
        bounds.append(axis_bounds)

    # to make this more even start facing up and add a right turn at the beginning
    mod_code = 'R' + code

    turn_indices = [i for i, c in enumerate(mod_code) if c == 'L' or c == 'R']
    moves = []

    for (i, j) in itertools.zip_longest(turn_indices, turn_indices[1:]):
        turn = mod_code[i]
        d = int(mod_code[i+1:j])

        moves.append((turn, d))

    return tiles, just_tiles, moves, bounds

In [4]:
ROTATE_LEFT = np.array([[0, -1], [1, 0]])

In [5]:
# tiles, just_tiles, moves, bounds = parse_p1()

In [6]:
def p1(s):
    tiles, just_tiles, moves, bounds = parse_p1(s)

    # it says right but we're doing up followed by an immediate right turn
    orientation = np.array([-1, 0])

    start_position = sorted(just_tiles)[0]
    position = start_position

    for turn, move in moves:
        assert move != 0

        # print(turn, move)
        # print(position, orientation)
        orientation = ((1 if turn == 'L' else -1) * ROTATE_LEFT) @ orientation
        # print(f"New orientation: {orientation}")

        # now check if we're wrapping if we walk in this direction

        move_axis = np.argwhere(orientation).item()
        # print(f"Move axis: {move_axis}")
        other_axis = 1 - move_axis
        sign = 1 if orientation[move_axis] > 0 else -1

        move_axis_position = position[move_axis]
        other_axis_position = position[other_axis]
        min_move, max_move = bounds[other_axis][other_axis_position]

        # print(f"Move bounds: {(min_move, max_move)}")

        for m in range(move):
            move_axis_next_position = (((move_axis_position - min_move + sign) % (max_move - min_move + 1)) + min_move)
            # print(move_axis_next_position, move_axis_position, sign, max_move, min_move)

            full_next_position = list(position)
            full_next_position[move_axis] = move_axis_next_position
            full_next_position = tuple(full_next_position)

            is_open = tiles[full_next_position]

            if not is_open:
                # print(f"Wall ahead at {full_next_position}")
                break
            else:
                position = full_next_position
                move_axis_position = move_axis_next_position
                # print(f"Moving forward. New position: {position}")
        
        # print()

        orient_x, orient_y = orientation

        if orient_x == 0 and orient_y == 1:
            orient_points = 0
        elif orient_x == 1 and orient_y == 0:
            orient_points = 1
        elif orient_x == 0 and orient_y == -1:
            orient_points = 2
        elif orient_x == -1 and orient_y == 0:
            orient_points = 3
        else:
            assert False
        
    return 1000 * (position[0] + 1) + 4 * (position[1] + 1) + orient_points

In [7]:
# p1(test_input)
p1(open("inputs/22").read())

3590

In [8]:
def euclid(a, b):
    q = a // b

    return q, (a - b*q)

In [9]:
def parse_p2(s, small_dim):
    board, code = s.split("\n\n")
    board_lines = board.split("\n")

    num_rows = len(board_lines)
    num_cols = max(len(b) for b in board_lines)

    small_boards = defaultdict(lambda: np.zeros((small_dim, small_dim), dtype=bool))

    for i, l in enumerate(board_lines):
        board_row, row_idx = euclid(i, small_dim)
        for j, c in enumerate(l):
            board_col, col_idx = euclid(j, small_dim)

            if c == ".":
                pass
            elif c == '#':
                small_boards[(board_row, board_col)][row_idx, col_idx] = 1
            else:
                pass

    sorted_boards = sorted(small_boards.items(), key=lambda x: x[0])
    ordered_boards = [v for k, v in sorted_boards]
    board_coords = [k for k, v in sorted_boards]

    # to make this more even start facing up and add a right turn at the beginning
    mod_code = 'R' + code

    turn_indices = [i for i, c in enumerate(mod_code) if c == 'L' or c == 'R']
    moves = []

    for (i, j) in itertools.zip_longest(turn_indices, turn_indices[1:]):
        turn = mod_code[i]
        d = int(mod_code[i+1:j])

        moves.append((turn, d))

    return moves, ordered_boards, board_coords

In [10]:
# small_dim = 4
# moves, ordered_boards, board_coords = parse_p2(test_input, small_dim)


small_dim = 50
moves, ordered_boards, board_coords = parse_p2(open("inputs/22").read(), small_dim)

In [12]:
# R D L U
# Edge correspondence for ordered boards

# my board
edges = [
    [0, 1, 2, 3],
    [4, 5, 0, 6],
    [5, 7, 8, 1],
    [9, 10, 2, 8],
    [4, 11, 9, 7],
    [11, 6, 3, 10],
]

# test board 
# edges = [
#     [0, 1, 2, 3],
#     [4, 5, 6, 3],
#     [7, 8, 4, 2],
#     [9, 10, 7, 1],
#     [11, 5, 8, 10],
#     [0, 6, 11, 9],
# ]

In [13]:
edge_to_edge = {}

In [14]:
# ugh
for i, g in itertools.groupby(sorted(np.ndenumerate(np.array(edges)), key=lambda x: x[1]), key=lambda x: x[1]):
    (ta, ea), (tb, eb) = g

    edge_to_edge[ta] = tb
    edge_to_edge[tb] = ta

In [15]:
orientation_to_dir = bijection({
    (0, 1): 0,
    (1, 0): 1,
    (0, -1): 2,
    (-1, 0): 3,
})

In [16]:
lower_limits = np.array([0, 0])
upper_limits = np.array([small_dim-1, small_dim-1])

In [17]:
def position_transition(position, dir, other_dir):
    # Transferring position (what next step best boundary is, in given direction)
    # orientation is always normal to edge direction
    SIGNS = [1, -1, -1, 1]

    flip = SIGNS[dir] * SIGNS[other_dir] > 0

    # R D L U

    # let N = small_dim - 1

    # if (R, R) then (a, N) to (N-a, N)
    # if (D, R) then (N, a) to (a, N)
    # if (L, R) then (a, 0) to (a, N)
    # if (U, R) then (0, a) to (N-a, N)

    # if (R, D) then (a, N) to (N, a)
    # if (D, D) then (N, a) to (N, N-a)
    # if (L, D) then (a, 0) to (N, N-a)
    # if (U, D) then (0, a) to (N, a)

    # if (R, L) then (a, N) to (a, 0)
    # if (D, L) then (N, a) to (N-a, 0)
    # if (L, L) then (a, 0) to (N-a, 0)
    # if (U, L) then (0, a) to (a, 0)

    # if (R, U) then (a, N) to (0, N-a)
    # if (D, U) then (N, a) to (0, a)
    # if (L, U) then (a, 0) to (0, a)
    # if (U, U) then (0, a) to (0, N-a)

    # flip the free direction if the sign matches
    # obviously alternating
    fixed_vals = [small_dim-1, small_dim-1, 0, 0]

    free_axis_start = dir % 2
    free_val_start = position[free_axis_start]

    fixed_axis_end = (other_dir+1)%2

    free_val_end = small_dim - 1 - free_val_start if flip else free_val_start
    fixed_val_end = fixed_vals[other_dir]

    next_position = np.zeros_like(position)
    next_position[fixed_axis_end] = fixed_val_end
    next_position[1-fixed_axis_end] = free_val_end

    return next_position

In [18]:
# it says right but we're doing up followed by an immediate right turn
orientation = np.array([-1, 0])

board = 0

# upper left position in first ordered board which isn't a wall
position = np.array(sorted([tuple(p) for p in np.transpose(np.where(~ordered_boards[board]))])[0])

for turn, move in moves:
    # turn
    orientation = ((1 if turn == 'L' else -1) * ROTATE_LEFT) @ orientation

    for m in range(move):
        # try to move
        new_position = position + orientation
        in_bounds = np.all(new_position <= upper_limits) and np.all(new_position >= lower_limits)

        # checking if next position is in bounds
        if in_bounds:
            # stay on board
            next_board = board
            next_orientation = orientation
            next_position = new_position
        else:
            # switch board
            dir = orientation_to_dir[tuple(orientation)] 
            next_board, other_dir = edge_to_edge[board, dir]

            # now compute the new orientation and position
            # always opposite of other_dir (180)
            next_orientation = ROTATE_LEFT @ ROTATE_LEFT @ np.array(orientation_to_dir.inverse[other_dir])
            next_position = position_transition(position, dir, other_dir)

        # is this * the right way? or use tuple? hm
        if ordered_boards[next_board][*next_position]:
            # rock, stop trying to move and don't update position etc
            break
        else:
            # update position, board, and orientation
            board = next_board
            orientation = next_orientation
            position = next_position

In [19]:
board_coord = np.array(board_coords[board])

((board_coord*small_dim + position + 1) * np.array([1000, 4])).sum() + orientation_to_dir[tuple(orientation)]

86382